In [1]:
import orjson
from datetime import datetime
import re
import pandas as pd

In [2]:
# with open("./data/AllIdentifiers.json", "r") as f:
#     all_identifiers = orjson.loads(f.read())["data"]

# with open("./data/AllPrintings.json", "r") as f:
#     all_printings = orjson.loads(f.read())["data"]

with open("./data/default-cards-20240520210708.json", "r") as f:
    default_cards = orjson.loads(f.read())

In [3]:
def get_sort_key(card):
    release_date = datetime.fromisoformat(card['released_at']) if card['released_at'] else datetime.max
    try:
        parsed_number = int(re.sub(r"[^\d]+", "", card['collector_number']))
    except ValueError as err:
        parsed_number = 0
    return release_date, card['set'], parsed_number, card['collector_number']

In [4]:
power_toughness_store = {}
for card in default_cards:
    power = card['power'] if 'power' in card else None
    toughness = card['toughness'] if 'toughness' in card else None
    if power is None and toughness is None:
        continue
    store_key = (power, toughness)
    if store_key in power_toughness_store:
        if get_sort_key(card) < get_sort_key(power_toughness_store[store_key]):
            power_toughness_store[store_key] = card
    else:
        power_toughness_store[store_key] = card

In [5]:
df = pd.DataFrame.from_dict(power_toughness_store, orient="index")
with pd.option_context('display.max_rows', None):
    display(df[['name', 'released_at']].sort_values('released_at'))

,,name,released_at
3,3,Northern Paladin,1993-08-05
5,5,Sea Serpent,1993-08-05
8,8,Force of Nature,1993-08-05
5,4,Water Elemental,1993-08-05
4,3,Pirate Ship,1993-08-05
*,*,Nightmare,1993-08-05
5,3,Juggernaut,1993-08-05
4,1,Phantasmal Forces,1993-08-05
0,2,Verduran Enchantress,1993-08-05
2,5,Veteran Bodyguard,1993-08-05


In [16]:
def generalize_mana_cost(mana_cost):
    """
    >>> generalize_mana_cost("{2}")
    '{2}'
    >>> generalize_mana_cost("{W}{W}")
    '{M}{M}'
    >>> generalize_mana_cost("{W}{U}{R}")
    '{M}{N}{O}'
    >>> generalize_mana_cost("{W}{U}{B}{R}{G}")
    '{W}{U}{B}{R}{G}'
    >>> generalize_mana_cost("{2/W}{2/U}")
    '{2/M}{2/N}'
    >>> generalize_mana_cost("{W/P}{W/U}{2/W}")
    '{M/P}{M/N}{2/M}'
    """
    generics = iter("MNOP")
    color_map = {}
    
    for c in mana_cost:
        if c in "WUBRG" and c not in color_map:
            try:
                color_map[c] = next(generics)
            except StopIteration:
                return mana_cost

    return mana_cost.translate(mana_cost.maketrans(color_map))

import doctest
doctest.testmod(verbose=True)

Trying:
    generalize_mana_cost("{2}")
Expecting:
    '{2}'
ok
Trying:
    generalize_mana_cost("{W}{W}")
Expecting:
    '{M}{M}'
ok
Trying:
    generalize_mana_cost("{W}{U}{R}")
Expecting:
    '{M}{N}{O}'
ok
Trying:
    generalize_mana_cost("{W}{U}{B}{R}{G}")
Expecting:
    '{W}{U}{B}{R}{G}'
ok
Trying:
    generalize_mana_cost("{2/W}{2/U}")
Expecting:
    '{2/M}{2/N}'
ok
Trying:
    generalize_mana_cost("{W/P}{W/U}{2/W}")
Expecting:
    '{M/P}{M/N}{2/M}'
ok
2 items had no tests:
    __main__
    __main__.get_sort_key
1 items passed all tests:
   6 tests in __main__.generalize_mana_cost
6 tests in 3 items.
6 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=6)